In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import Model


In [2]:
#Electrodes:Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10
electrodes_list = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8','FC5','FC1','FC2','FC6','T7','C3','Cz','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10','P7','P3','Pz','P4','P8','PO9','O1','Oz','O2','PO10']

In [3]:
spr_data = open('D:\CSAI Y2\S2\ResWork\spr_data\dbc_spr_data_combined.csv', 'r')
data = open('D:\CSAI Y2\S2\ResWork\eeg_data\data\dbc_data.csv', 'r')

electrodes = {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
averages =  {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
electrodes_copy = {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}
averages_copy =  {'Fp1':[],'Fp2':[],'F7':[],'F3':[],'Fz':[],'F4':[],'F8':[],'FC5':[],'FC1':[],'FC2':[],'FC6':[],'T7':[],'C3':[],'Cz':[],'C4':[],'T8':[],'TP9':[],'CP5':[],'CP1':[],'CP2':[],'CP6':[],'TP10':[],'P7':[],'P3':[],'Pz':[],'P4':[],'P8':[],'PO9':[],'O1':[],'Oz':[],'O2':[],'PO10':[]}


In [4]:
#Save the values for electrodes in dictionary
for line in data:
    line1 = line.split(sep=',')
    for i, el in zip(range(5, 37), electrodes.keys()):
        electrodes[el].append(line1[i])
#eliminate the header
for el in electrodes.keys():
    electrodes[el].pop(0)


In [5]:
print(type(electrodes['Fp1'][0]))
print(electrodes['Fp1'][:5])
#convert to float
for key in electrodes.keys():
    electrodes[key] = [float(x) for x in electrodes[key]]
        

<class 'str'>
['-1.20971', '-0.83101', '-0.11259', '0.89073', '2.05291']


In [6]:
#average of each electrode
for key, key1 in zip(electrodes.keys(), averages.keys()):
    averages[key1] = sum(electrodes[key])/len(electrodes[key])
averages


{'Fp1': -0.05310859048392986,
 'Fp2': 0.2718770229591897,
 'F7': 0.4739086527210903,
 'F3': 0.8584300532158233,
 'Fz': 0.9760052670377233,
 'F4': 0.8981730826762923,
 'F8': -0.00890235936146882,
 'FC5': 0.7833091127319093,
 'FC1': 1.4234224100262434,
 'FC2': 0.875951609840779,
 'FC6': 0.46008143968769044,
 'T7': 0.7198228696197088,
 'C3': 0.9743223610622136,
 'Cz': 1.1573413757112458,
 'C4': 1.0664270036023475,
 'T8': -0.027104529298083095,
 'TP9': -0.818397348600777,
 'CP5': 0.8979413112554142,
 'CP1': 1.3363747745206653,
 'CP2': 1.4031188761749551,
 'CP6': 0.964616609570151,
 'TP10': -0.6674093293521983,
 'P7': -0.16842819519170188,
 'P3': 1.4784557713975757,
 'Pz': 1.8039131734925908,
 'P4': 1.5727795646336864,
 'P8': 0.9878751239332164,
 'PO9': 0.9215345374613586,
 'O1': 1.2220036146876843,
 'Oz': 1.6918947176793908,
 'O2': 1.7964585402674693,
 'PO10': 1.069962982575769}

Time window avarage prediction
Example: 
Subject 5 had RT of 1183ms (Control) -> use that to predict the avg in the time window 400-1183 for electrode Fp1

First, store the data

In [7]:
response_time = []
for line in spr_data:
    line1 = line.split(sep = ',')
    response_time.append(line1[3])
response_time.pop(0)
print(response_time)

['841', '754', '1197', '351', '1051', '538', '1191', '772', '877', '687', '816', '649', '502', '606', '1032', '1731', '592', '1061', '1183', '1547', '791', '1218', '928', '474', '482', '361', '1086', '654', '346', '842', '717', '686', '678', '1014', '1450', '813', '1048', '497', '1074', '526', '690', '567', '473', '538', '908', '493', '1546', '448', '1083', '691', '1178', '1978', '792', '863', '674', '1233', '470', '855', '601', '605', '640', '804', '893', '758', '730', '482', '539', '756', '1389', '876', '568', '620', '736', '436', '651', '844', '548', '1241', '664', '701', '319', '396', '613', '826', '273', '765', '1110', '456', '805', '1544', '810', '758', '582', '501', '786', '855', '500', '563', '777', '268', '678', '243', '758', '1003', '642', '723', '971', '495', '910', '834', '1032', '678', '873', '1132', '685', '1511', '1362', '732', '1647', '1792', '580', '608', '999', '473', '878', '967', '1140', '508', '574', '897', '786', '468', '983', '643', '445', '482', '744', '1073', '

In [8]:
spr_data = pd.read_csv('D:\CSAI Y2\S2\ResWork\spr_data\dbc_spr_data_combined.csv')
data = pd.read_csv("D:\CSAI Y2\S2\ResWork\eeg_data\data\dbc_data.csv")
    

In [9]:
data.head(5)

,ItemNum,Condition,Subject,Timestamp,TrialNum,Fp1,Fp2,F7,F3,Fz,...,P8,PO9,O1,Oz,O2,PO10,VEOG,HEOG,Assoc,Plaus
0,1,control,5,-200,197,-1.20971,2.52400,-5.24529,-3.35825,-5.09060,...,2.29405,0.64024,-4.89769,-6.64080,-3.25357,-0.07328,0.71351,1.91076,6.8,6.416667
1,1,control,5,-198,197,-0.83101,2.92082,-5.41510,-3.22667,-4.61091,...,2.45947,0.78223,-5.05034,-6.61679,-2.91444,0.10294,0.67929,2.78725,6.8,6.416667
2,1,control,5,-196,197,-0.11259,3.49500,-4.92985,-2.57258,-3.57396,...,2.82271,1.14153,-4.55636,-5.89756,-2.12281,0.54867,0.59286,3.42723,6.8,6.416667
3,1,control,5,-194,197,0.89073,4.20807,-3.93422,-1.44027,-2.04021,...,3.32898,1.65022,-3.49408,-4.58794,-0.94843,1.23226,0.41795,3.69663,6.8,6.416667
4,1,control,5,-192,197,2.05291,4.89957,-2.72258,0.02966,-0.17849,...,3.87386,2.19716,-2.01769,-2.85254,0.51508,2.09137,0.10579,3.52454,6.8,6.416667


In [10]:
spr_data.head(5)

,ItemNum,Condition,Subject,RT,Assoc,Plaus,Accuracy
0,1,control,19,841,6.8,6.416667,1
1,1,control,10,754,6.8,6.416667,1
2,1,control,13,1197,6.8,6.416667,1
3,1,control,16,351,6.8,6.416667,1
4,1,control,7,1051,6.8,6.416667,1


In [21]:
Sum = 0
data_main = dict() 
for row, row1 in zip(data.iterrows(), spr_data.iterrows()):
    if row[1]['Subject'] == row[1]['Subject'] and row[1]['Condition'] == row[1]['Condition']:
        if (row[1]['Subject'], row[1]["Condition"]) not in data_main:
            data_main[(row[1]['Subject'], row[1]["Condition"])] = electrodes_copy

        for i in electrodes_list:
            data_main[(row[1]['Subject'], row[1]["Condition"])][i].append(row[1][i]]) 

    

In [22]:
data_main

{(5,
  'control'): {'Fp1': [-1.20971,
   -0.83101,
   -0.11259,
   0.89073,
   2.05291,
   3.23713,
   4.32897,
   5.21275,
   5.76257,
   5.89135,
   5.57964,
   4.84867,
   3.74861,
   2.38774,
   0.93443,
   -0.42453,
   -1.49535,
   -2.06261,
   -1.9579,
   -1.20739,
   -0.08372,
   1.01227,
   1.73528,
   1.93946,
   1.69654,
   1.19621,
   0.66137,
   0.3052,
   0.25084,
   0.44875,
   0.71856,
   0.89582,
   0.91205,
   0.74894,
   0.3599,
   -0.34196,
   -1.4071,
   -2.76793,
   -4.2368,
   -5.58126,
   -6.62922,
   -7.3354,
   -7.77067,
   -8.05114,
   -8.24127,
   -8.28697,
   -8.02788,
   -7.28391,
   -5.98403,
   -4.27098,
   -2.45546,
   -0.81436,
   0.55223,
   1.66545,
   2.53057,
   3.10089,
   3.36003,
   3.37103,
   3.21939,
   2.95599,
   2.61315,
   2.23299,
   1.84214,
   1.42676,
   0.99138,
   0.62086,
   0.40992,
   0.34513,
   0.32283,
   0.25867,
   0.1202,
   -0.10394,
   -0.39833,
   -0.70646,
   -0.95588,
   -1.08058,
   -1.02694,
   -0.79655,
   -0.52136,
